In [1]:
import os
import numpy as np
import cloudknot as ck

In [2]:
slices = [1, 2, 3]
videos = [1, 2, 3, 4, 5]
func = ['PEG', 'COOH']

files = []
start_knot = 8
for slic in slices:
    for video in videos:
        for fun in func:
            files.append('1000nm_{}_P1_S{}_XY{}.tif'.format(fun, slic, video))

In [3]:
def bin_images(file):
    import diff_classifier.aws as aws
    import skimage.io as sio
    from skimage import transform as tf
    import numpy as np
    
    rfolder = 'Tissue_Studies/04_30_18_JBSS_in_Tissue'
    rfile = '{}/{}'.format(rfolder, file)
    aws.download_s3(rfile, file, bucket_name='ccurtis.data')
    
    def tiff_resize(fname, factor=10):
        massive = sio.imread(fname)
        dims = massive.shape
        small = tf.resize(massive, (dims[0], dims[1]//factor, dims[2]//factor))
        nfname = '{}_sm.tif'.format(fname.split('.')[0])
        
        sio.imsave(nfname, small)
        return nfname, small
    
    nfname, small = tiff_resize(file, factor=4)
    print('Successfully binned image')
    
    rnfile = '{}/{}'.format(rfolder, nfname)
    aws.upload_s3(nfname, rnfile, bucket_name='ccurtis.data')

In [4]:
github_installs=('https://github.com/ccurtis7/diff_classifier.git')
my_image = ck.DockerImage(func=bin_images, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

In [5]:
#Test Docker Image
#Change skimage to scikit-image and change boto3 version to 1.5.28
my_image.build("0.1", image_name="test_image")

In [6]:
knot = ck.Knot(name='bin_images_{}'.format(start_knot),
               docker_image = my_image,
               memory = 100000,
               resource_type = 'SPOT',
               bid_percentage = 100,
               image_id = 'ami-0de34a0a338c1051b',
               pars_policies=('AmazonS3FullAccess',))
              
result_futures = knot.map(files)

In [ ]:
knot.clobber()

In [ ]:
import diff_classifier.aws as aws
import skimage.io as sio
from skimage import transform as tf
import numpy as np
file = files[0]
rfolder = 'Tissue_Studies/04_30_18_JBSS_in_Tissue'
rfile = '{}/{}'.format(rfolder, file)
aws.download_s3(rfile, file, bucket_name='ccurtis.data')

In [ ]:
def tiff_resize(fname, factor=10):
    massive = sio.imread(fname)
    dims = massive.shape
    small = np.moveaxis(tf.resize(massive, (dims[0], dims[1]//factor, dims[2]//factor)), 0, -1)
    nfname = '{}_sm.tif'.format(fname.split('.')[0])

    sio.imsave(nfname, small)
    return nfname, small

In [ ]:
massive = sio.imread(file)

In [ ]:
test = np.moveaxis(massive, 0, -1)
test2 = test[0:100, 0:100, 0:10]

In [ ]:
test3 = test[0:500, 0:500, 0:10]

In [ ]:
test3.shape

In [ ]:
sio.imsave('test.tif', massive[:, 0:100, 0:100])

In [ ]:
nfname, small = tiff_resize('test.tif', factor=4)

In [ ]:
fname = 'test.tif'
factor = 4
massive = sio.imread(fname)
dims = massive.shape

In [ ]:
small = np.moveaxis(tf.resize(massive, (dims[0], dims[1]//factor, dims[2]//factor)), 0, -1)
nfname = '{}_sm.tif'.format(fname.split('.')[0])

sio.imsave(nfname, small)

In [ ]:
tiff_resize('test.tif', factor=4)

In [ ]:
2048/4